In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the CSV file into a DataFrame
df_ibex = pd.read_csv('data/ibex.csv')

# Convert the 'date' column to datetime if it's not already in datetime format
df_ibex['Date'] = pd.to_datetime(df_ibex['Date'])

# Set the 'date' column as the index
df_ibex.set_index('Date', inplace=True)

# Drop 'Volume' and 'Adj Close' columns
df_ibex = df_ibex.drop(['Volume', 'Adj Close'], axis=1)

In [3]:
df_ibex

,Open,High,Low,Close
Date,,,,
2011-01-03,9899.400391,9993.599609,9850.500000,9888.299805
2011-01-04,9895.500000,9974.200195,9799.400391,9888.400391
2011-01-05,9877.500000,9882.400391,9599.299805,9801.400391
2011-01-06,9803.200195,9832.599609,9678.400391,9702.700195
2011-01-07,9682.400391,9699.400391,9497.799805,9560.700195
...,...,...,...,...
2019-12-23,9650.200195,9670.900391,9639.000000,9659.599609
2019-12-24,9632.099609,9661.799805,9607.799805,9661.799805
2019-12-27,9673.000000,9700.500000,9657.500000,9700.500000


In [4]:
# Define the split date
split_date = pd.to_datetime('2019-01-01')

# Split the data into train and test sets
df_ibex_train = df_ibex[df_ibex.index < split_date]
df_ibex_test = df_ibex[df_ibex.index >= split_date]

In [5]:
def ema(df, period):
    return df.ewm(span = period, min_periods = period).mean() 

In [6]:
def df_change(df, symbol):
    temp = pd.DataFrame(df['Close'])
    temp = temp.rename({'Close': symbol}, axis=1)
    return temp

In [7]:
def tema_strategy(df, symbol, short_period, mid_period, long_period):
    df = df_change(df, symbol)
    temp = df.copy()
    temp['ShortEMA'] = ema(df, short_period)
    temp['MidEMA'] = ema(df, mid_period)
    temp['LongEMA'] = ema(df, long_period)
    temp['Position'] = np.where((temp['ShortEMA'] > temp['MidEMA']) & (temp['ShortEMA'] > temp['LongEMA']), 1,
                                np.where((temp['ShortEMA'] < temp['MidEMA']) & (temp['ShortEMA'] < temp['LongEMA']), -1, 0))
    temp['LogReturns'] = np.log(temp[symbol]) - np.log(temp[symbol]).shift(1)
    temp['Strategy'] = temp.Position.shift(1) * temp['LogReturns']
    temp['Trades'] = temp.Position.diff().fillna(0).abs()
    temp['CLogReturns'] = temp['LogReturns'].cumsum().apply(np.exp)
    temp["CStrategy"] = temp['Strategy'].cumsum().apply(np.exp)
    return temp

In [54]:
from itertools import product
import heapq

def optimize_parameters(df, symbol, short_range, mid_range, long_range):
    top_combinations = []
    
    for short_period, mid_period, long_period in product(short_range, mid_range, long_range):
        if short_period < mid_period < long_period:
            result = tema_strategy(df, symbol, short_period, mid_period, long_period)
            c_strategy = result['CStrategy'].iloc[-1]
            
            # Use a heap to maintain the top 5 combinations based on CStrategy
            if len(top_combinations) < 5:
                heapq.heappush(top_combinations, (c_strategy, (short_period, mid_period, long_period)))
            else:
                heapq.heappushpop(top_combinations, (c_strategy, (short_period, mid_period, long_period)))
    
    # Get the top 5 combinations with highest CStrategy values
    top_combinations = sorted(top_combinations, reverse=True)
    best_combinations = [combo[1] for combo in top_combinations]
    best_c_strategy = [combo[0] for combo in top_combinations]
    
    return best_combinations, best_c_strategy

In [55]:
# Define the parameter ranges
short_range = range(5, 11)
mid_range = range(6, 16)
long_range = range(10, 21)

# Run the optimization
best_combinations, best_c_strategy = optimize_parameters(df_ibex_train, 'IBEX', short_range, mid_range, long_range)

# Print the results
print("Best Combinations:")
for i, combination in enumerate(best_combinations):
    print(f"Combination {i+1}: {combination}")
print("Best CStrategy:", best_c_strategy)

Best Combinations:
Combination 1: (5, 6, 18)
Combination 2: (5, 6, 19)
Combination 3: (5, 6, 14)
Combination 4: (5, 6, 17)
Combination 5: (5, 6, 13)
Best CStrategy: [0.6598617734695321, 0.6586041390127836, 0.6320741090457743, 0.6270812715605683, 0.6105537839891231]


In [58]:
best_params = best_combinations[0]

In [59]:
tema_strategy(df_ibex_train, 'IBEX', best_params[0], best_params[1], best_params[2])

,IBEX,ShortEMA,MidEMA,LongEMA,Position,LogReturns,Strategy,Trades,CLogReturns,CStrategy
Date,,,,,,,,,,
2011-01-03,9888.299805,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN
2011-01-04,9888.400391,NaN,NaN,NaN,0,0.000010,0.000000,0.0,1.000010,1.000000
2011-01-05,9801.400391,NaN,NaN,NaN,0,-0.008837,-0.000000,0.0,0.991212,1.000000
2011-01-06,9702.700195,NaN,NaN,NaN,0,-0.010121,-0.000000,0.0,0.981230,1.000000
2011-01-07,9560.700195,9700.224392,NaN,NaN,0,-0.014743,-0.000000,0.0,0.966870,1.000000
...,...,...,...,...,...,...,...,...,...,...
2018-12-21,8556.799805,8668.619282,8687.297424,8817.041671,-1,-0.004629,0.004629,0.0,0.865346,0.658559
2018-12-24,8480.599609,8605.946057,8628.240905,8781.626717,-1,-0.008945,0.008945,0.0,0.857640,0.664476
2018-12-27,8363.900391,8525.264169,8552.715044,8737.655525,-1,-0.013856,0.013856,0.0,0.845838,0.673747


In [60]:
def calculate_profit_factor(strategy):
    positive_trades = strategy.loc[strategy > 0].sum()
    negative_trades = strategy.loc[strategy < 0].sum()
    profit_factor = abs(positive_trades / negative_trades)
    return profit_factor

In [61]:
calculate_profit_factor(tema_strategy(df_ibex_train, 'IBEX', best_params[0], best_params[1], best_params[2])['Strategy'])

0.9513374498343222